<h1 id=\"tocheading\">Table of Contents</h1>
    <div id=\"toc\"></div>

In [3]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

// needed to generate the Table of contents 
// taken from github.com/kmahelona/ipython_notebook_goodies

<IPython.core.display.Javascript object>

# Modeling

## Fit Models with Training Data Set

## Review Model Outcomes — Iterate over additional models as needed.

## Identify the Final Model

In [5]:
import pandas as pd
import numpy as np


In [ ]:
#initialize dict to collect model performance data
models_comparison = {'Model': [],
        'Explained Variance': [],
        'Mean Absolute Error': [],
        'Features Dropped': []
        }

In [4]:
#Using it again:
X_train_scaled = pd.read_csv("data/X_train_scaled.csv")
X_test_scaled = pd.read_csv("data/X_test_scaled.csv")

feature_list = pd.read_csv("data/feature_list.csv")
feature_list = pd.Index(list(feature_list["0"]))

# add_cols = list(feature_list.difference(X_test_scaled.columns))

# drop_cols = list(X_test_scaled.columns.difference(feature_list))

# for col in add_cols:
#     X_test_scaled[col] = np.nan

# for col in drop_cols:
#     X_test_scaled = X_test_scaled.drop(col, axis = 1)

# # reorder columns
# X_test_scaled = X_test_scaled[feature_list]

# types = pd.read_csv("data/data_types.csv")
# for i in range(len(types)):
#     X_test_scaled[types.iloc[i,0]] = X_test_scaled[types.iloc[i,0]].astype(types.iloc[i,1])
# #Make Predictions

# scaler = joblib.load("scaler.pkl")
# #predictions = regressor.predict(X_test)
# #Save Prediction Results

# res = pd.DataFrame(predictions)
# res.index = X_test.index # its important for comparison
# res.columns = ["prediction"]
# res.to_csv("prediction_results.csv")

NameError: name 'pd' is not defined